In [10]:
import ee
import geemap
import numpy as np
import pandas as pd
import folium
import copy
import math
from tqdm import trange,tqdm
from Basic_tools import *
from functools import partial
import geopandas as gpd
geemap.set_proxy(port=10809)
# ee.Authenticate()
ee.Initialize()
print('geemap version = {}\ngeemap path = {}'.format(geemap.__version__,geemap.__path__))

geemap version = 0.20.3
geemap path = ['D:\\Code_base\\anaconda\\envs\\GEE\\lib\\site-packages\\geemap']


In [27]:
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/Wu_Asia_Southest_GL_wgs84').filter(ee.Filter.gte('GL_Area', 0.1))
    #projects/ee-mrwurenzhe/assets/Glacial_lake/Checkout_polygen

# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()
})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)       # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

In [ ]:
START_DATE  = ee.Date('2019-08-01')
END_DATE   = ee.Date('2019-08-30')

DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)

models = ['volume', 'surface', None]
Buffer = 0
Model = models[0]
I = 1143
scale = 30

AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(I))
s1_col = (ee.ImageCollection("COPERNICUS/S1_GRD")
          .filter(ee.Filter.eq('instrumentMode', 'IW'))
            .filterBounds(AOI)
            .filterDate(START_DATE, END_DATE))
AOI_buffer = AOI.buffer(distance=10000).bounds()

In [62]:
s1_descending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')).first()
s1_ascending = s1_col.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')).first()

image = s1_ascending
orbitProperties_pass = image.get('orbitProperties_pass').getInfo()
# get the image geometry and projection
geom = image.geometry()
proj = image.select(1).projection()

def getASCCorners(image, AOI_buffer):
    # 真实方位角
    coords = ee.Array(image.geometry().coordinates().get(0)).transpose()
    crdLons = ee.List(coords.toList().get(0))
    crdLats = ee.List(coords.toList().get(1))
    minLon = crdLons.sort().get(0)
    maxLon = crdLons.sort().get(-1)
    minLat = crdLats.sort().get(0)
    maxLat = crdLats.sort().get(-1)
    azimuth = (ee.Number(crdLons.get(crdLats.indexOf(minLat))).subtract(minLon).atan2(ee.Number(crdLats.get(crdLons.indexOf(minLon))).subtract(minLat))
               .multiply(180.0 / math.pi))

    if orbitProperties_pass == 'ASCENDING':
        azimuth = azimuth.add(270.0)
        rotationFromNorth = azimuth.subtract(360.0)
    elif orbitProperties_pass == 'DESCENDING':
        azimuth = azimuth.add(180.0)
        rotationFromNorth = azimuth.subtract(180.0)
    else:
        raise TypeError
    azimuthEdge = (ee.Feature(ee.Geometry.LineString([crdLons.get(crdLats.indexOf(minLat)), minLat, minLon,
                              crdLats.get(crdLons.indexOf(minLon))]), {'azimuth': azimuth}).copyProperties(image))

    coords = ee.Array(image.clip(
        AOI_buffer).geometry().coordinates().get(0)).transpose()
    crdLons = ee.List(coords.toList().get(0))
    crdLats = ee.List(coords.toList().get(1))
    minLon = crdLons.sort().get(0)
    maxLon = crdLons.sort().get(-1)
    minLat = crdLats.sort().get(0)
    maxLat = crdLats.sort().get(-1)
    azimuth = (ee.Number(crdLons.get(crdLats.indexOf(minLat))).subtract(minLon).atan2(ee.Number(crdLats.get(crdLons.indexOf(minLon))).subtract(minLat))
               .multiply(180.0 / math.pi))
    if orbitProperties_pass == 'ASCENDING':
        # 左上角
        startpoint = ee.List([minLon, maxLat])
        # 右下角
        endpoint = ee.List([maxLon, minLat])
    elif orbitProperties_pass == 'DESCENDING':
        # 右上角
        startpoint = ee.List([maxLon, maxLat])
        # 左下角
        endpoint = ee.List([minLon, minLat])

    coordinates_dict = {'crdLons': crdLons, 'crdLats': crdLats,
                        'minLon': minLon, 'maxLon': maxLon, 'minLat': minLat, 'maxLat': maxLat}

    return azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict

Angle_aspect = ee.Terrain.aspect(image.select('angle'))
# s1_azimuth_across = Angle_aspect.reduceRegion(ee.Reducer.mean(), geom, 1000).get('aspect')
azimuthEdge, rotationFromNorth, startpoint, endpoint, coordinates_dict = getASCCorners(image, AOI_buffer)
Heading = azimuthEdge.get('azimuth')
Heading_Rad = ee.Image.constant(Heading).multiply(np.pi / 180)

s1_azimuth_across = ee.Number(Heading).subtract(90.0)
theta_iRad = image.select('angle').multiply(np.pi / 180)  # 地面入射角度转为弧度
phi_iRad = ee.Image.constant(s1_azimuth_across).multiply(np.pi / 180)  # 方位角转弧度

def slop_aspect(elevation,proj,geom):
    alpha_sRad = ee.Terrain.slope(elevation).select('slope').multiply(
        np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡度(与地面夹角)
    phi_sRad = ee.Terrain.aspect(elevation).select('aspect').multiply(
        np.pi / 180).setDefaultProjection(proj).clip(geom)          # 坡向角，(坡度陡峭度)坡与正北方向夹角(陡峭度)，从正北方向起算，顺时针计算角度
    phi_rRad = phi_iRad.subtract(phi_sRad)     # (飞行方向角度-坡度陡峭度)飞行方向与坡向之间的夹角
    # 分解坡度，在水平方向和垂直方向进行分解，为固定公式，cos对应水平分解，sin对应垂直分解
    alpha_rRad = (alpha_sRad.tan().multiply(phi_rRad.cos())).atan()     # 距离向分解
    alpha_azRad = (alpha_sRad.tan().multiply(phi_rRad.sin())).atan()     # 方位向分解
    return alpha_sRad,phi_sRad,alpha_rRad,alpha_azRad

alpha_sRad_SRTM,phi_sRad_SRTM,alpha_rRad_SRTM,alpha_azRad_SRTM = slop_aspect(DEMSRTM,proj,geom)
alpha_sRad_NASA,phi_sRad_NASA,alpha_rRad_NASA,alpha_azRad_NASA = slop_aspect(DEMNASA,proj,geom)
alpha_sRad_ALOS,phi_sRad_ALOS,alpha_rRad_ALOS,alpha_azRad_ALOS = slop_aspect(DEMALOS,proj,geom)
alpha_sRad_COPERNICUS,phi_sRad_COPERNICUS,alpha_rRad_COPERNICUS,alpha_azRad_COPERNICUS = slop_aspect(DEMCOPERNICUS,proj,geom)

In [ ]:
def _volumetric_model_SCF(theta_iRad, alpha_rRad):
    '''Code for calculation of volumetric model SCF
        体积模型
    :param theta_iRad: ee.Image of incidence angle in radians
    :param alpha_rRad: ee.Image of slope steepness in range

    :returns: ee.Image
    '''
    # create a 90 degree image in radians
    ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

    # model
    nominator = (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).tan()
    denominator = (ninetyRad.subtract(theta_iRad)).tan()
    return nominator.divide(denominator)

def _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad):
    '''Code for calculation of direct model SCF
        表面模型
    :param theta_iRad: ee.Image of incidence angle in radians
    :param alpha_rRad: ee.Image of slope steepness in range
    :param alpha_azRad: ee.Image of slope steepness in azimuth
    :returns: ee.Image
    '''
    # create a 90 degree image in radians
    ninetyRad = ee.Image.constant(90).multiply(np.pi / 180)

    # model
    nominator = (ninetyRad.subtract(theta_iRad)).cos()
    denominator = (alpha_azRad.cos().multiply(
        (ninetyRad.subtract(theta_iRad).add(alpha_rRad)).cos()))

    return nominator.divide(denominator)

In [63]:
model = 'volume'
sigma0Pow = ee.Image.constant(10).pow(image.divide(10.0))
gamma0 = sigma0Pow.divide(theta_iRad.cos())                                                                # 根据角度修订入射值 
gamma0dB = ee.Image.constant(10).multiply(gamma0.log10()).select(['VV', 'VH'], ['VV_gamma0', 'VH_gamma0']) # 根据角度修订入射值
ratio_gamma = (gamma0dB.select('VV_gamma0').subtract(gamma0dB.select('VH_gamma0')).rename('ratio_gamma0')) # gamma极化相减

def volumetric(model,theta_iRad, alpha_rRad,alpha_azRad):
    if model == 'volume':
        scf = _volumetric_model_SCF(theta_iRad, alpha_rRad)
    if model == 'surface':
        scf = _surface_model_SCF(theta_iRad, alpha_rRad, alpha_azRad)
        
    gamma0_flat = gamma0.divide(scf)
    gamma0_flatDB = (ee.Image.constant(10).multiply(gamma0_flat.log10()).select(['VV', 'VH'],['VV_gamma0flat', 'VH_gamma0flat']))
    ratio_flat = (gamma0_flatDB.select('VV_gamma0flat').subtract(
        gamma0_flatDB.select('VH_gamma0flat')).rename('ratio_gamma0flat'))
    return scf,gamma0_flat,gamma0_flatDB,ratio_flat

volumetric_dict = {}
if orbitProperties_pass == 'ASCENDING':
    volumetric_dict['SRTM'] = volumetric(model,theta_iRad, alpha_rRad_SRTM,alpha_azRad_SRTM)
    volumetric_dict['NASA'] = volumetric(model,theta_iRad, alpha_rRad_NASA,alpha_azRad_NASA)
    volumetric_dict['ALOS'] = volumetric(model,theta_iRad, alpha_rRad_ALOS,alpha_azRad_ALOS)
    volumetric_dict['COPE'] = volumetric(model,theta_iRad, alpha_rRad_COPERNICUS,alpha_azRad_COPERNICUS)
    ASCENDING_dict = copy.deepcopy(volumetric_dict)
elif orbitProperties_pass == 'DESCENDING':
    volumetric_dict['SRTM'] = volumetric(model,theta_iRad, alpha_rRad_SRTM,alpha_azRad_SRTM)
    volumetric_dict['NASA'] = volumetric(model,theta_iRad, alpha_rRad_NASA,alpha_azRad_NASA)
    volumetric_dict['ALOS'] = volumetric(model,theta_iRad, alpha_rRad_ALOS,alpha_azRad_ALOS)
    volumetric_dict['COPE'] = volumetric(model,theta_iRad, alpha_rRad_COPERNICUS,alpha_azRad_COPERNICUS)
    DESCENDING_dict = copy.deepcopy(volumetric_dict)

- DEM的优劣性
- alpha_sRad不存在负值，alpha_rRad存在负值，原因是什么
- 体模型和面模型，辐射斜率校正会不会受到负值影响
- 校正过后联合升降轨道，如果无法直接联合，考虑使用标准化或者归一化图像

## 采用NASADEM做测试，开展升降轨道融合

In [26]:
s1_descending_VV = copy.deepcopy(image.select('VV'))
s1_descending_VV_gammaflat = copy.deepcopy(gamma0_flatDB_NASA.select('VV_gamma0flat'))

## 查看alpha_s \ alpha_r的分布,观察在山体两侧角度值，不同DEM的差异

In [5]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(AOI, zoom=14)
Map.addLayer(DEMSRTM.clip(geom) ,{'min':3644,'max':5970},'DEMSRTM')
Map.addLayer(DEMNASA.clip(geom) ,{'min':3644,'max':5970},'DEMNASA')
Map.addLayer(DEMALOS.clip(geom) ,{'min':3644,'max':5970},'DEMALOS')
Map.addLayer(DEMCOPERNICUS.clip(geom) ,{'min':3644,'max':5970},'DEMCOPERNICUS')
Map

Map(center=[30.17633989163365, 94.28168536055013], controls=(WidgetControl(options=['position', 'transparent_b…

In [64]:
# 查看辐射斜率校正前后图像变化
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(AOI, zoom=14)
Map.addLayer(s1_descending.select('VV'),{'min':-18,'max':5},'s1_descending')
Map.addLayer(DESCENDING_dict['SRTM'][2].select('VV_gamma0flat') ,{'min':-18,'max':5},'DESCENDING_dict')
Map.addLayer(s1_ascending.select('VV'),{'min':-18,'max':5},'s1_ascending')
Map.addLayer(ASCENDING_dict['SRTM'][2].select('VV_gamma0flat') ,{'min':-18,'max':5},'ASCENDING_dict')
Map

Map(center=[30.17633989163365, 94.28168536055013], controls=(WidgetControl(options=['position', 'transparent_b…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.

In [17]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(AOI, zoom=14)
Map.addLayer(alpha_rRad_SRTM, {'min':0,'max':1.5}, 'alpha_rRad_SRTM')
Map.addLayer(alpha_sRad_SRTM, {'min':-1.4,'max':1.5}, 'alpha_sRad_SRTM')
Map.addLayer(alpha_rRad_NASA, {'min':0,'max':1.5}, 'alpha_rRad_NASA')
Map.addLayer(alpha_sRad_NASA, {'min':-1.4,'max':1.5}, 'alpha_sRad_NASA')
Map.addLayer(alpha_rRad_ALOS, {'min':0,'max':1.5}, 'alpha_rRad_ALOS')
Map.addLayer(alpha_sRad_ALOS, {'min':-1.4,'max':1.5}, 'alpha_sRad_ALOS')
Map.addLayer(alpha_rRad_COPERNICUS, {'min':0,'max':1.5}, 'alpha_rRad_COPERNICUS')
Map.addLayer(alpha_sRad_COPERNICUS, {'min':-1.4,'max':1.5}, 'alpha_sRad_COPERNICUS')
Map

Map(center=[30.17633989163365, 94.28168536055013], controls=(WidgetControl(options=['position', 'transparent_b…

In [65]:
get_minmax(ASCENDING_dict['SRTM'][2])

In [66]:
get_minmax(DESCENDING_dict['SRTM'][2])

In [34]:
Map = geemap.Map(basemap='HYBRID')
Map.centerObject(AOI, zoom=14)
Map.addLayer(scf_SRTM, {}, 'scf_SRTM')
Map.addLayer(scf_NASA, {}, 'scf_NASA')
Map.addLayer(scf_ALOS, {}, 'scf_ALOS')
Map.addLayer(scf_COPERNICUS, {}, 'scf_COPERNICUS')
Map

Map(center=[30.17633989163365, 94.28168536055013], controls=(WidgetControl(options=['position', 'transparent_b…

TraitError: The 'east' trait of a Map instance expected a float, not the NoneType None.